In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import preprocessing

In [2]:
# Import dataset with panda
df = pd.read_csv('output.csv')
features_considered = ['embb_req', 'embb_true', 'urllc_req', 'urllc_true','miot_req', 'miot_true','mmtc_req', 'mmtc_true','voice_req', 'voice_true']
features = df[features_considered]
features.index = df['round']
features.head()


,embb_req,embb_true,urllc_req,urllc_true,miot_req,miot_true,mmtc_req,mmtc_true,voice_req,voice_true
round,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0
1,48366352451,14753425662,821443044,821443044,454899066,454899066,412873975,412873975,789671880,789671880
2,41635005196,15327079419,366694134,366694134,259996062,259996062,184881966,184881966,737354839,737354839
3,33702013813,15105126775,455571105,455571105,255653544,255653544,292088284,292088284,600314013,600314013
4,33635062725,14973135597,798957171,798957171,422170133,422170133,400555854,400555854,471688801,471688801


In [3]:
# Parametes 
TRAIN_SPLIT = 64000
tf.random.set_seed(13)
past_history = 1
future_target = 0
step = 1
BATCH_SIZE = 32 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 80000 # >= len dataset for perfetct shuffling
EVALUATION_INTERVAL = 2000 # steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
VALIDATION_STEPS = 500 # validation_steps = TotalvalidationSamples / ValidationBatchSize
EPOCHS = 20
bs_capacity = 20000000000


In [4]:
# Normalize
dataset = features.values
dataset = dataset / bs_capacity


In [5]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)


In [6]:
# Create validation and training set
x_train, y_train = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], TRAIN_SPLIT, None, past_history, future_target, step)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [7]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

# Compile RNN
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

# Train RNN
history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 2000 steps, validate for 500 steps
Epoch 1/20
2000/2000 [==============================] - 13s 6ms/step - loss: 0.0109 - val_loss: 0.0063
Epoch 2/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0068 - val_loss: 0.0053
Epoch 3/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0063 - val_loss: 0.0052
Epoch 4/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0061 - val_loss: 0.0045
Epoch 5/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0059 - val_loss: 0.0051
Epoch 6/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0058 - val_loss: 0.0045
Epoch 7/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0057 - val_loss: 0.0046
Epoch 8/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0056 - val_loss: 0.0047
Epoch 9/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0055 - val_loss: 0.0041
Epoch 10/20
2000/2000 [=====

In [8]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[28117743919, 554516117, 283582751, 309281138, 510198997]]]
test = np.array(test)/bs_capacity

prediction = model.predict(test)
print(prediction[0])
print(y_train[2]) # because I took the input of the first values in x_train for test


[0.83397067 0.03898463 0.02037413 0.02118398 0.02022251]
[0.83236394 0.03206446 0.01918653 0.01900058 0.02111502]
[0.8454119  0.03891478 0.02013643 0.02121386 0.02078872]
[0.74865678 0.03994786 0.02110851 0.02002779 0.02358444]


In [9]:
# Save trained model
model.save("model.h5")
